# Capstone project, Part 3
for Zoomcamp, Ahmed Yahia Kallel

A continuation of the notebook, with focus on Ensemble-based and sophisticated regressors.

So far, these are the best output
* GB-lr003nest1000
* training accuracy: 0.494942
* validation accuracy: 0.544177

# Train.py (part of)

In [61]:

# libarires
import numpy as np
import pandas as pd

#splitter
from sklearn.model_selection import train_test_split

#pipeline
from sklearn.pipeline import Pipeline


from sklearn.ensemble import VotingRegressor

#transformers
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.preprocessing import *


#model
from sklearn.ensemble import GradientBoostingRegressor

#save pickle
import pickle

#read database
df = pd.read_csv("Property Prices in Tunisia.csv") #load file

#translation
df.replace(df["category"].unique(), ['Land and Farms', 'Apartments', 'Holiday rentals',
       'Shops, Businesses and Industrial Premises', 'Houses and Villas',
       'Flatshare', 'Offices and Trays'],inplace=True)

df.replace(df["type"].unique(), ['For sale', 'For rent'],inplace=True)

df.replace("Autres villes","Others", inplace=True)

del df["price"] 


# data splitting

#split according to random state = 1
df_trainval, df_test = train_test_split(df, test_size=0.2, random_state=1) #split train+val [80%], test [20%]
df_train, df_val = train_test_split(df_trainval, test_size=0.25, random_state=1) #split train[60%] val [20%]

#reset, drop index
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

y_test = df_test["log_price"].values
y_train = df_train["log_price"].values
y_val = df_val["log_price"].values

del df_test["log_price"]
del df_train["log_price"]
del df_val["log_price"]



# feature types
feat_cat = ['category','type','city','region']
feat_num = ['room_count','bathroom_count','size']

## transform

In [62]:
transformations = [
    ('Scal_num', RobustScaler(unit_variance=True) , feat_num),
    ('categorical', OneHotEncoder(dtype=np.int32,handle_unknown = 'ignore'), feat_cat)
]
transf = ColumnTransformer(transformations, remainder='drop')



libs:

In [63]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.wrappers.scikit_learn import KerasRegressor

#models
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor

In [64]:
def rms(v):
    return np.sqrt(np.mean(v**2))

non keras pipelines:

In [65]:
pipeline_linear_regression = Pipeline([
    ('transformer', transf),
    ('lr', LinearRegression())
])

pipeline_svr = Pipeline([
    ('transformer', transf),
    ('svr', SVR(C=0.9,epsilon=0.15))
])

pipeline_rfr = Pipeline([
    ('transformer', transf),
    ('rfr', RandomForestRegressor(max_depth=5, n_estimators=31))
])
pipeline_enet = Pipeline([
    ('transformer', transf),
    ('en', ElasticNet(alpha=0.004, l1_ratio=0.055))
])
pipeline_gb = Pipeline([
    ('transformer', transf),
    ('gb', GradientBoostingRegressor(learning_rate=0.03,n_estimators=1000 ))
])

Keras pipeline:

In [66]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1,use_bias=True))
    opt = keras.optimizers.Adamax()
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model
kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
kr._estimator_type = "regressor"
pipeline_kr = Pipeline([
        ('transformer', transf),
        ('nn', kr)
    ])


C:\Users\kay_t\AppData\Local\Temp/ipykernel_60204/841995247.py:7: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)


# Testing Voting Regressor

In [69]:
voting1 = VotingRegressor(
             estimators=[('lr', pipeline_linear_regression),
                         ('svr', pipeline_svr),
                         ('rfr',pipeline_rfr),
                         ('enet',pipeline_enet),
                         ('gb',pipeline_gb),
                         ('kr',pipeline_kr)])
voting1.fit(df_train,y_train)
y_train_hat = voting1.predict(df_train)
y_val_hat = voting1.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))


C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_4/dense_4/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_4/dense_4/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_4/dense_4/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Training accuracy (RMSE): 0.510
Validation accuracy (RMSE): 0.542


In [68]:
voting2 = VotingRegressor(
             estimators=[('lr', pipeline_linear_regression),
                         ('svr', pipeline_svr),
                         ('rfr',pipeline_rfr),
                         ('enet',pipeline_enet),
                         ('gb',pipeline_gb)])
#  seems like it doesn't provide much addition
voting2.fit(df_train,y_train)
y_train_hat = voting2.predict(df_train)
y_val_hat = voting2.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))


Training accuracy (RMSE): 0.507
Validation accuracy (RMSE): 0.541


# Testing AdaBoostRegressor

In [80]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression

pipeline_adab = Pipeline([
    ('transformer', transf),
    ('ada', AdaBoostRegressor(random_state=0, n_estimators=100,learning_rate=0.01, base_estimator=GradientBoostingRegressor(learning_rate=0.03,n_estimators=1000 )))
])

pipeline_adab.fit(df_train,y_train)
y_train_hat = pipeline_adab.predict(df_train)
y_val_hat = pipeline_adab.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))

Training accuracy (RMSE): 0.495
Validation accuracy (RMSE): 0.549


In [87]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression

pipeline_adab = Pipeline([
    ('transformer', transf),
    ('ada', AdaBoostRegressor(random_state=0, n_estimators=200,learning_rate=0.0001, base_estimator=GradientBoostingRegressor( )))
])

pipeline_adab.fit(df_train,y_train)
y_train_hat = pipeline_adab.predict(df_train)
y_val_hat = pipeline_adab.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))

Training accuracy (RMSE): 0.516
Validation accuracy (RMSE): 0.544


In [89]:
# it took around 23 mins
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression

pipeline_adab = Pipeline([
    ('transformer', transf),
    ('ada', AdaBoostRegressor(random_state=0, n_estimators=200,learning_rate=0.0001, base_estimator=SVR( )))
])

pipeline_adab.fit(df_train,y_train)
y_train_hat = pipeline_adab.predict(df_train)
y_val_hat = pipeline_adab.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))

KeyboardInterrupt: 

# BaggingRegressor

In [93]:
# it took around 23 mins
from sklearn.ensemble import BaggingRegressor

pipeline_br = Pipeline([
    ('transformer', transf),
    ('br', BaggingRegressor(random_state=0, n_estimators=1000, base_estimator=GradientBoostingRegressor( )))
])

pipeline_br.fit(df_train,y_train)
y_train_hat = pipeline_br.predict(df_train)
y_val_hat = pipeline_br.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))

Training accuracy (RMSE): 0.513
Validation accuracy (RMSE): 0.544


# Extra Trees Regressor

In [100]:
from sklearn.ensemble import ExtraTreesRegressor

pipeline_etr = Pipeline([
    ('transformer', transf),
    ('et', ExtraTreesRegressor(random_state=0, n_estimators=1000,  max_features='sqrt',n_jobs=16,warm_start=True,bootstrap=True))
])

pipeline_etr.fit(df_train,y_train)
y_train_hat = pipeline_etr.predict(df_train)
y_val_hat = pipeline_etr.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))

Training accuracy (RMSE): 0.372
Validation accuracy (RMSE): 0.557


In [108]:
from sklearn.ensemble import ExtraTreesRegressor

pipeline_etr = Pipeline([
    ('transformer', transf),
    ('et', ExtraTreesRegressor(random_state=0, n_estimators=1000,  max_features='auto',n_jobs=16,warm_start=True,bootstrap=True,min_samples_split=7))
])

pipeline_etr.fit(df_train,y_train)
y_train_hat = pipeline_etr.predict(df_train)
y_val_hat = pipeline_etr.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))

Training accuracy (RMSE): 0.422
Validation accuracy (RMSE): 0.545


# Final Voting Regressor

In [109]:
voting3 = VotingRegressor(
             estimators=[('lr', pipeline_linear_regression),
                         ('svr', pipeline_svr),
                         ('rfr',pipeline_rfr),
                         ('enet',pipeline_enet),
                         ('br',pipeline_br),
                         ('etr',pipeline_etr)])
#  seems like it doesn't provide much addition
voting3.fit(df_train,y_train)
y_train_hat = voting3.predict(df_train)
y_val_hat = voting3.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))


Training accuracy (RMSE): 0.491
Validation accuracy (RMSE): 0.538


In [111]:
# save pipeline_voting as best method
with open("pipeline_voting.bin", 'wb') as f:  pickle.dump(voting3,f)

# Stacking Regressor

In [116]:
from sklearn.ensemble import StackingRegressor

estimators=[('lr', pipeline_linear_regression),
                         ('svr', pipeline_svr),
                         ('rfr',pipeline_rfr),
                         ('enet',pipeline_enet),
                         ('br',pipeline_br),
                         ('etr',pipeline_etr),
                         ('gb',pipeline_gb)]

pipeline_stack = StackingRegressor(estimators=estimators,n_jobs=16,cv=3)

pipeline_stack.fit(df_train,y_train)
y_train_hat = pipeline_stack.predict(df_train)
y_val_hat = pipeline_stack.predict(df_val)
print('Training accuracy (RMSE): %2.3f' % rms(y_train_hat-y_train))
print('Validation accuracy (RMSE): %2.3f' % rms(y_val_hat-y_val))

Training accuracy (RMSE): 0.481
Validation accuracy (RMSE): 0.535


In [117]:
with open("pipeline_stacking.bin", 'wb') as f:  pickle.dump(pipeline_stack,f)

# Testing

Please use [extra_predictors/predict_stacking.py](extra_predictors/predict_stacking.py) or [extra_predictors/predict_voting.py](extra_predictors/predict_voting.py)  for fast-testing

In [140]:
# client.py: get data of the new user
import requests
import json


id =201 #select here an id to test!
example = df_test.iloc[id:id+1,:].to_json()
expected_res = y_test[id]



url = "http://127.0.0.1:9696/predict"
results = requests.post(url, json=example).json()
print(df_test.iloc[id:id+1,:])
log_price = results["log_price"]


print("Price for ID %d : ~%d TND [log_price: %2.3f]" % (id, 10**log_price, log_price))
#print("actual price: %d TND [log_price: %2.3f]" % (10**expected_res,expected_res))


# rounded
#print("Price for ID %d : ~%d TND [log_price: %2.3f]" % (id, 10**round(log_price,1), round(log_price,1)))
#print("actual price: %d TND [log_price: %2.3f]" % (10**round(expected_res,1),round(expected_res,1)))


       category  room_count  bathroom_count   size      type    city  region
201  Apartments         2.0             1.0  110.0  For sale  Mahdia  Mahdia
Price for ID 201 : ~187029 TND [log_price: 5.272]
